# Predictive analysis of naval incidents in the USA, 2002 - 2015: <br>
## Annex 5.1. Data Model VesselBalancedSample

> Author: [Oscar Anton](https://www.linkedin.com/in/oscanton/) <br>
> Date: 2024 <br>
> License: [CC BY-NC-ND 4.0 DEED](https://creativecommons.org/licenses/by-nc-nd/4.0/) <br>
> Version: 0.9 <br>

## 0. Loadings

### Libraries

In [ ]:
# System environment
import os

# Data general management
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import graphviz

# Model training
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, ShuffleSplit, learning_curve
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

# Model metrics
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, f1_score, mean_absolute_error, roc_auc_score, roc_curve, auc, cohen_kappa_score, confusion_matrix
from scipy.stats import ttest_rel

# Model Export
import joblib

# Model explainers
import dalex as dx

### General variables

In [ ]:
# Main data folders
merged_activity_folder = '../3.DataPreprocess/DataMergedActivity'
datasets_folder = 'Datasets'
models_folder = 'Models'

# Toggle for export data to external file
file_export_enabled = False
# Toggle for train model
train_model_enabled = False

# Available CPU cores for multiprocessing (training models)
n_jobs = os.cpu_count() - 1
# Random seed for reproducibility
seed = 42

### Base dataframe

In [ ]:
# Load dataframe
VesselBalancedSample = pd.read_feather(merged_activity_folder + '/' + 'VesselBalancedSample.feather')

# Check dataframe
VesselBalancedSample.head(5)

# 1. Dataframe creation

## 1.1. Variable transformation

> Note: Missing and NA values previously processed. Data balanced by design

### 1.1.1. Category variables: Reduce excessive variability

In [ ]:
# Function: cutting values
def cut_years(column):
    return pd.cut(column,
                  bins=[-float('inf'), 1940, 1960, 1980, 2000, float('inf')],
                  labels=['very Old', 'old', 'average', 'new', 'very new'],
                  include_lowest=True)

# Function: group minority values 
def lump_factorials(column, prop=0.008, other_level="other value"):
    counts = column.value_counts(normalize=True)
    mask = column.isin(counts[counts < prop].index)
    column[mask] = other_level
    return column

# First transformations: assign 'y': No event = 0, otherwise = 1
# Cutting, renaming, dropping not relevant variables
data_general = (
    VesselBalancedSample
    .assign(y=lambda x: pd.factorize(x['event_type'] != "No event", sort=True)[0])
    .assign(build_year=cut_years(VesselBalancedSample['build_year']))
    .rename(columns={'length': 'vessel_length'})
    .drop(columns=['vessel_id', 'imo_number', 'vessel_name', 'event_type', 'damage_status'])
)

# Group minority values 
data_general[['vessel_class', 'flag_abbr', 'classification_society', 'solas_desc']] = (data_general[['vessel_class', 'flag_abbr', 'classification_society', 'solas_desc']]
    .apply(lump_factorials)
)

# Check structure
data_general.dtypes

### 1.1.2. Category variables: one-hot-encoding

In [ ]:
# One hot encoding for not numeric variables
data_ohe = (pd.get_dummies(data_general
                .select_dtypes(exclude=['number']))
                .astype(int))

### 1.1.3. Numeric variables: Scale

In [ ]:
# Apply standard scaler
data_scaled = pd.DataFrame(StandardScaler()
               .fit_transform(data_general[['gross_ton', 'vessel_length']])
)

# Rename column names as strings
data_scaled.columns = ['gross_ton', 'vessel_length']

## 1.2. Join data

In [ ]:
# Dataset join: encoded, scaled and 'y'
data_num = pd.concat([data_ohe,
                     data_scaled,
                     data_general['y']],
                     axis=1)

# Rename column names as strings
data_num.columns = data_num.columns.astype(str)

# Verify variables
for column in data_num.columns:
    print(f"Name: {column} | Type: {data_num[column].dtype} | Levels: {data_num[column].nunique()}")

In [ ]:
# Save dataset
if file_export_enabled :
    data_num.reset_index().to_feather(datasets_folder + '/' + 'data_num.feather')
    print(f'data_num {data_num.shape} exported to {datasets_folder}')
else:
    data_num = pd.read_feather(datasets_folder + '/' + 'data_num.feather')
    print(f'data_num {data_num.shape} imported from {datasets_folder}')

## 1.3. Train / Test Split

In [ ]:
# Data split
X_train, X_test, y_train, y_test = train_test_split(
                                    data_num.drop(columns=['y']),
                                    data_num['y'],
                                    test_size=0.2,
                                    random_state=42)

In [ ]:
# Save / Load dataframes in one file (h5 format for multiple data)
if file_export_enabled :
    dfs = {'X_train':X_train, 'X_test':X_test, 'y_train':y_train, 'y_test':y_test}
    for key, df in dfs.items():
        df.to_hdf(datasets_folder + '/' + 'datasets_splited.h5', key=key, format='table')
        print(f'{key} {eval(key).shape} exported to {datasets_folder} folder') 
else:
    dfs = ['X_train', 'X_test', 'y_train', 'y_test']
    for df in dfs:
        globals()[df] = pd.read_hdf(datasets_folder + '/' + 'datasets_splited.h5', key = df)
        print(f'{df} {eval(df).shape} imported from {datasets_folder} folder') 

# 2. Model Training

## Performance displaying functions

In [ ]:
# Function: Table with main metrics data
def model_metrics(model, X, y):
    # Predictions (absolute)
    y_pred = model.predict(X)
    
    # Calculate main metrics
    roc_auc = round(roc_auc_score(y, y_pred), 4)
    accuracy = round(accuracy_score(y, y_pred), 4)
    kappa = round(cohen_kappa_score(y, y_pred), 4)
    rmse = round(mean_squared_error(y, y_pred), 4)
    mae = round(mean_absolute_error(y, y_pred), 4)
    r2 = round(r2_score(y, y_pred), 4)
    f1 = round(f1_score(y, y_pred), 4)

    # Sensitivity And Specificity
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    sensitivity =  round(tp / (tp + fn), 4)
    specificity = round(tn / (tn + fp), 4)
    
    # Build multiindex table
    metrics_df = pd.DataFrame([['ROC AUC:', roc_auc],['Accuracy:', accuracy], ['Kappa:', kappa],
                            ['RMSE:', rmse], ['MAE:', mae], ['R2:', r2], ['F1:', f1],[' ', ' '],
                            ['Sensitivity:', sensitivity], ['Specificity:', specificity]],
                            columns=pd.MultiIndex.from_product([[model.__class__.__name__],['Metric', 'Value']]))
    
    return metrics_df.style.hide()

In [ ]:
# Function: Table with Confusion Matrix data
def confusion_matrix_table(model, X, y):
    # Predictions (absolute)
    y_pred = model.predict(X)

    # Confusion matrix
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()

    # Dataframe creation
    df = pd.DataFrame([[tp, fn],[fp, tn]],
                  index=pd.Index(['1', '0'], name='Actual Label:'),
                  columns=pd.MultiIndex.from_product([[model.__class__.__name__],['1', '0']], names=['Model:', 'Predicted:']))

    # Dataframe style
    styled_df = df.style.set_table_styles([
        {'selector': 'th.col_heading', 'props': 'text-align: center;'},
        {'selector': 'td', 'props': 'text-align: center;'},
    ], overwrite=False)

    return styled_df

In [ ]:
# Function: Plot ROC Curve
def plot_roc_curve(model, X, y):
    # Predicted probabilities
    y_score = model.predict_proba(X)
    
    # Calculate ROC for each class
    fpr, tpr, _ = roc_curve(y, y_score[:, 1])
    
    # Calculate AUC (Area Under Curve)
    roc_auc = auc(fpr, tpr)
    
    # Plot ROC Curve
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='black', lw=1, linestyle='dotted')
    plt.xlim([0, 1])
    plt.ylim([0, 1.01])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve for {model.__class__.__name__}')
    plt.legend(loc="lower right")
    plt.show()

## 2.1. Bayesian networks

### 2.1.1. Naïve Bayes -Gaussian- (NB)

#### Model Training

In [ ]:
if train_model_enabled :
    # Define model parameters
    params = {}                         # No parameters for this model

    # Create and train model
    model = GaussianNB(**params).fit(X_train, y_train)

    # Save to external file
    joblib.dump(model, models_folder + '/' + 'nb_train.pkl')
    
else:
    # Load model from external file
    model = joblib.load(models_folder + '/' + 'nb_train.pkl')

# Get model parameters and print as a one row list
print(f'Model name: {model.__class__.__name__} \nParameters:')
params = model.get_params()
for param_name, param_value in params.items():
    print(f" {param_name}: {param_value}")

#### Model Main Metrics

In [ ]:
# Call function to show main metrics
model_metrics(model, X_test, y_test)

In [ ]:
# Call function to show confusion matrix
confusion_matrix_table(model, X_test, y_test)

In [ ]:
# Call function to plot Receiver Operating Characteristic (ROC) curve
plot_roc_curve(model, X_test, y_test)

### 2.1.2. Bagged Naïve Bayes -Gaussian- (BNB)

#### Model Training

In [ ]:
if train_model_enabled :
    # Define model parameters
    params = {}                         # No parameters for this model

    # Create and train model. 
    model = BaggingClassifier(GaussianNB(**params), n_estimators=5).fit(X_train, y_train)

    # Save to external file
    joblib.dump(model, models_folder + '/' + 'bnb_train.pkl')

else:
    # Load model from external file
    model = joblib.load(models_folder + '/' + 'bnb_train.pkl')

# Bagging (Bootstrap Aggregating) is a technique used to improve the stability and accuracy.
# n_estimators specifies how many individual Naive Bayes classifiers are trained independently on subsets of the training data.
# It approximates the behavior of AODE
    
# Get model parameters and print as a one row list
print(f'Model name: {model.__class__.__name__} \nParameters:')
params = model.get_params()
for param_name, param_value in params.items():
    print(f" {param_name}: {param_value}")

#### Model Main Metrics

In [ ]:
# Call function to show main metrics
model_metrics(model, X_test, y_test)

In [ ]:
# Call function to show confusion matrix
confusion_matrix_table(model, X_test, y_test)

In [ ]:
# Call function to plot Receiver Operating Characteristic (ROC) curve
plot_roc_curve(model, X_test, y_test)

## 2.2. Gradient Boosting Models

### 2.2.1. Gradient Boosting Machine (GBM)

#### Model Training

In [ ]:
if train_model_enabled :
    # Define model parameters
    params = {
        'n_estimators': 500,            # Number of trees
        'learning_rate': 0.1,           # Contribution of each tree to the model
        'max_depth': 3,                 # Maximum levels of each tree
        'random_state': seed,           # Random seed for reproducibility
    }

    # Create and train model
    model = GradientBoostingClassifier(**params).fit(X_train, y_train)

    # Save to external file
    joblib.dump(model, models_folder + '/' + 'GBM_train.pkl')

else:
    # Load model from external file
    model = joblib.load(models_folder + '/' + 'GBM_train.pkl')

# Get model parameters and print as a one row list
print(f'Model name: {model.__class__.__name__} \nParameters:')
params = model.get_params()
for param_name, param_value in params.items():
    print(f" {param_name}: {param_value}")

#### Model Main Metrics

In [ ]:
# Call function to show main metrics
model_metrics(model, X_test, y_test)

In [ ]:
# Call function to show confusion matrix
confusion_matrix_table(model, X_test, y_test)

In [ ]:
# Call function to plot Receiver Operating Characteristic (ROC) curve
plot_roc_curve(model, X_test, y_test)

#### Extra: Main metrics per iteration (n_estimators=500)

In [ ]:
# Number of iterations during training
iterations = int(model.get_params().get("n_estimators") + 1)

# Obtain prediction probabilities at each stage of training
train_probs = np.array([preds[:, 1] for preds in model.staged_predict_proba(X_train)])
test_probs = np.array([preds[:, 1] for preds in model.staged_predict_proba(X_test)])

# Calculate ROC metric at each stage of training
train_roc_auc = [roc_auc_score(y_train, prob) for prob in train_probs]
test_roc_auc = [roc_auc_score(y_train, prob) for prob in train_probs]

# Plot the ROC metric as a function of iterations
plt.plot(range(1, iterations), train_roc_auc, label="Training ROC AUC", color="blue")
plt.plot(range(1, iterations), test_roc_auc, label="Test ROC AUC", color="red")
plt.xlabel("Number of Iterations")
plt.ylabel("ROC AUC")
plt.title("ROC AUC Metric per iteration")
plt.legend()
plt.show()

In [ ]:
# Get predictions at each stage of training
train_errors = [accuracy_score(y_train, y_pred) for y_pred in model.staged_predict(X_train)]
test_errors = [accuracy_score(y_test, y_pred) for y_pred in model.staged_predict(X_test)]

# Plot the Accuracy metric as a function of iterations
plt.plot(np.arange(1, iterations), train_errors, label="Training Accuracy", color="blue")
plt.plot(np.arange(1, iterations), test_errors, label="Test Accuracy", color="red")
plt.xlabel("Number of Iterations")
plt.ylabel("Accuracy")
plt.title("Accuracy Metric per iteration")
plt.legend()
plt.show()

In [ ]:
# Get predictions at each stage of training
train_kappa = [cohen_kappa_score(y_train, y_pred) for y_pred in model.staged_predict(X_train)]
test_kappa = [cohen_kappa_score(y_test, y_pred) for y_pred in model.staged_predict(X_test)]

# Plot the Kappa metric as a function of iterations
plt.plot(np.arange(1, iterations), train_kappa, label="Training Kappa", color="blue")
plt.plot(np.arange(1, iterations), test_kappa, label="Test Kappa", color="red")
plt.xlabel("Number of Iterations")
plt.ylabel("Kappa")
plt.title("Kappa Metric per iteration")
plt.legend()
plt.show()

In [ ]:
sensitivities = []
specificities = []

# Get values per each iteration 
for i, y_pred in enumerate(model.staged_predict(X_test)):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Plot sensitivity and specificity for each iteration of training
plt.figure(figsize=(10, 6))
plt.plot(range(1, iterations), sensitivities, label='Sensitivity', color='blue')
plt.plot(range(1, iterations), specificities, label='Specificity', color='green')
plt.xlabel('Number of Estimators')
plt.ylabel('Value')
plt.title('Sensitivity and Specificity per Training Iteration')
plt.legend()
plt.grid(True)
plt.show()

### 2.2.2. Extreme Gradient Boosting (XGB)

#### Model Training

In [ ]:
if train_model_enabled :
    # Define model parameters
    params = {
    'objective': 'binary:logistic',     # Binary classification problem
    'eval_metric': 'error',             # Evaluation metric: error rate
    'eta': 0.1,                         # Learning rate
    'max_depth': 3,                     # Maximum tree depth
    'colsample_bytree': 0.3,            # Subsample ratio of columns when constructing each tree
    'random_state': seed                # Random seed for reproducibility
    }

    # Create and train model
    model = XGBClassifier(**params).fit(X_train, y_train)

    # Save to external file
    joblib.dump(model, models_folder + '/' + 'XGB_train.pkl')

else:
    # Load model from external file
    model = joblib.load(models_folder + '/' + 'XGB_train.pkl')

# Get model parameters and print as a one row list
print(f'Model name: {model.__class__.__name__} \nParameters:')
params = model.get_params()
for param_name, param_value in params.items():
    print(f" {param_name}: {param_value}")

#### Model Main Metrics

In [ ]:
# Call function to show main metrics
model_metrics(model, X_test, y_test)

In [ ]:
# Call function to show confusion matrix
confusion_matrix_table(model, X_test, y_test)

In [ ]:
# Call function to plot Receiver Operating Characteristic (ROC) curve
plot_roc_curve(model, X_test, y_test)

## 2.3. More models

### 2.3.1. Random Forest (RF)

#### Model Training

In [ ]:
if train_model_enabled :   
    # Define model parameters
    params = {
        'n_estimators': 100,            # Number of trees in the forest
        'max_depth': None,              # Maximum depth of the trees (no restrictions)
        'random_state': seed            # Random seed for reproducibility
    }

    # Create and train model
    model = RandomForestClassifier(**params).fit(X_train, y_train)

    # Save to external file
    joblib.dump(model, models_folder + '/' + 'rf_train.pkl')

else:
    # Load model from external file
    model = joblib.load(models_folder + '/' + 'rf_train.pkl')

# Get model parameters and print as a one row list
print(f'Model name: {model.__class__.__name__} \nParameters:')
params = model.get_params()
for param_name, param_value in params.items():
    print(f" {param_name}: {param_value}")

#### Model Main Metrics

In [ ]:
# Call function to show main metrics
model_metrics(model, X_test, y_test)

In [ ]:
# Call function to show confusion matrix
confusion_matrix_table(model, X_test, y_test)

In [ ]:
# Call function to plot Receiver Operating Characteristic (ROC) curve
plot_roc_curve(model, X_test, y_test)

#### Extra: Main metrics per iteration (n_estimators=100)

In [ ]:
# Define the number of iterations, initial number of estimators, and step size
n_iterations = 10
initial_n_estimators = 10
step_size = 10

# Create empty lists to store the number of estimators and ROC AUC scores
num_estimators_list = []
roc_auc_scores = []

# Iterate over the specified number of iterations
for i in range(n_iterations):
    # Create the Random Forest model with the current number of estimators
    model = RandomForestClassifier(n_estimators=initial_n_estimators + i * step_size, random_state=seed)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions and calculate ROC AUC score
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    # Store the number of estimators and ROC AUC score
    num_estimators_list.append(initial_n_estimators + i * step_size)
    roc_auc_scores.append(roc_auc)

# Plot the ROC AUC scores over the number of estimators
plt.figure(figsize=(10, 6))
plt.plot(num_estimators_list, roc_auc_scores, label="ROC AUC", color="red")
plt.title('ROC AUC Score vs. Number of Estimators')
plt.xlabel('Number of Estimators')
plt.ylabel('ROC AUC Score')
plt.grid(True)

In [ ]:
# Create empty lists to store the number of estimators and ROC AUC scores
num_estimators_list = []
accuracy_auc_scores = []

# Iterate over the specified number of iterations
for i in range(n_iterations):
    # Create the Random Forest model with the current number of estimators
    model = RandomForestClassifier(n_estimators=initial_n_estimators + i * step_size, random_state=seed)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Predictions
    y_pred = model.predict(X_test)

    # Calculate Accuracy scores
    accuracy_auc = accuracy_score(y_test, y_pred)
    
    # Store the number of estimators and Accuracy score
    num_estimators_list.append(initial_n_estimators + i * step_size)
    accuracy_auc_scores.append(accuracy_auc)

# Plot the Accuracy scores over the number of estimators
plt.figure(figsize=(10, 6))
plt.plot(num_estimators_list, accuracy_auc_scores, label="Accuracy", color="red")
plt.title('Accuracy Score vs. Number of Estimators')
plt.xlabel('Number of Estimators')
plt.ylabel('Accuracy Score')
plt.grid(True)

In [ ]:
# Create empty lists to store the number of estimators and ROC AUC scores
num_estimators_list = []
kappa_scores = []

# Iterate over the specified number of iterations
for i in range(n_iterations):
    # Create the Random Forest model with the current number of estimators
    model = RandomForestClassifier(n_estimators=initial_n_estimators + i * step_size, random_state=seed)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Predictions
    y_pred = model.predict(X_test)

    # Calculate Kappa scores
    kappa = cohen_kappa_score(y_test, y_pred)
    
    # Store the number of estimators and Kappa scores
    num_estimators_list.append(initial_n_estimators + i * step_size)
    kappa_scores.append(kappa)

# Plot the Kappa scores over the number of estimators
plt.figure(figsize=(10, 6))
plt.plot(num_estimators_list, accuracy_auc_scores, label="Kappa", color="red")
plt.title('Kappa Score vs. Number of Estimators')
plt.xlabel('Number of Estimators')
plt.ylabel('Kappa Score')
plt.grid(True)

#### Extra: Explore structure of a tree example

In [ ]:
if file_export_enabled :
    # Extract a random model tree (for example, the first one)
    tree = model.estimators_[0]

    # Export the tree to Graphviz
    dot_data = export_graphviz(tree, out_file=None, 
                                filled=True, rounded=True,  
                                special_characters=True, precision=5)  
    graph = graphviz.Source(dot_data)

    # Export to pdf file (lighter than svg)
    graph.render('rf_train_tree', format='pdf', cleanup=True)

# Note: File is not displayed because is extremely width

### 2.3.2. Support Vector Machine -Radial Kernel- (SVM)

#### Model Training

In [ ]:
if train_model_enabled :
    # Define model parameters
    params = {
        'kernel': 'rbf',                # Radial Kernel
        'probability': True,            # Enable probability estimates (uses 5-fold cross-validation)
        'random_state': seed            # Random seed for reproducibility
    }

    # Create and train model
    model = SVC(**params).fit(X_train, y_train)

    # Save to external file
    joblib.dump(model, models_folder + '/' + 'svm_train.pkl')

else:
    # Load model from external file
    model = joblib.load(models_folder + '/' + 'svm_train.pkl')

# Get model parameters and print as a one row list
print(f'Model name: {model.__class__.__name__} \nParameters:')
params = model.get_params()
for param_name, param_value in params.items():
    print(f" {param_name}: {param_value}")

#### Model Main Metrics

In [ ]:
# Call function to show main metrics
model_metrics(model, X_test, y_test)

In [ ]:
# Call function to show confusion matrix
confusion_matrix_table(model, X_test, y_test)

In [ ]:
# Call function to plot Receiver Operating Characteristic (ROC) curve
plot_roc_curve(model, X_test, y_test)

### 2.3.3. Multilayer Perceptron (MLP)

#### Model Training

In [ ]:
if train_model_enabled :
    # Define model parameters
    params = {
        'hidden_layer_sizes': (100, 50),    # Two hidden layers with 100 and 50 neurons respectively.
        'activation': 'relu',               # Activation function for the hidden layers: Rectified Linear Unit
        'solver': 'adam',                   # Optimization algorithm
        'random_state': seed                # Random seed for reproducibility
    }

    # Create and train model
    model = MLPClassifier(**params).fit(X_train, y_train)

    # Save to external file
    joblib.dump(model, models_folder + '/' + 'nnet_train.pkl')

else:
    # Load model from external file
    model = joblib.load(models_folder + '/' + 'nnet_train.pkl')

# Get model parameters and print as a one row list
print(f'Model name: {model.__class__.__name__} \nParameters:')
params = model.get_params()
for param_name, param_value in params.items():
    print(f" {param_name}: {param_value}")

#### Model Main Metrics

In [ ]:
# Call function to show main metrics
model_metrics(model, X_test, y_test)

In [ ]:
# Call function to show confusion matrix
confusion_matrix_table(model, X_test, y_test)

In [ ]:
# Call function to plot Receiver Operating Characteristic (ROC) curve
plot_roc_curve(model, X_test, y_test)

### 2.3.4. Classification and Regression Trees (CART)

#### Model Training

In [ ]:
if train_model_enabled :
    # Define model parameters
    params = {
        'criterion':'gini',                 # Criteria
        'random_state': seed                # Random seed for reproducibility
    }

    # Create and train model
    model = DecisionTreeClassifier(**params).fit(X_train, y_train)

    # Save to external file
    joblib.dump(model, models_folder + '/' + 'cart_train.pkl')

else:
    # Load model from external file
    model = joblib.load(models_folder + '/' + 'cart_train.pkl')

# Get model parameters and print as a one row list
print(f'Model name: {model.__class__.__name__} \nParameters:')
params = model.get_params()
for param_name, param_value in params.items():
    print(f" {param_name}: {param_value}")

#### Model Main Metrics

In [ ]:
# Call function to show main metrics
model_metrics(model, X_test, y_test)

In [ ]:
# Call function to show confusion matrix
confusion_matrix_table(model, X_test, y_test)

In [ ]:
# Call function to plot Receiver Operating Characteristic (ROC) curve
plot_roc_curve(model, X_test, y_test)

### 2.3.5. Logistic Regression

#### Model Training

In [ ]:
if train_model_enabled :
    # Define model parameters
    params = {
        'random_state': seed                # Random seed for reproducibility
    }

    # Create and train model
    model = LogisticRegression(**params).fit(X_train, y_train)

    # Save to external file
    joblib.dump(model, models_folder + '/' + 'rl_train.pkl')

else:
    # Load model from external file
    model = joblib.load(models_folder + '/' + 'rl_train.pkl')

# Get model parameters and print as a one row list
print(f'Model name: {model.__class__.__name__} \nParameters:')
params = model.get_params()
for param_name, param_value in params.items():
    print(f" {param_name}: {param_value}")

#### Model Main Metrics

In [ ]:
# Call function to show main metrics
model_metrics(model, X_test, y_test)

In [ ]:
# Call function to show confusion matrix
confusion_matrix_table(model, X_test, y_test)

In [ ]:
# Call function to plot Receiver Operating Characteristic (ROC) curve
plot_roc_curve(model, X_test, y_test)

# 3. Model Comparison

#### Load saved models

In [ ]:
# All model list
models = ['nb_train', 'bnb_train', 'GBM_train',
          'XGB_train', 'rf_train', 'svm_train',
          'nnet_train', 'cart_train', 'rl_train']

# Load all models
for model in models:
    globals()[model] = joblib.load(models_folder + '/' + model + '.pkl')
    print(f'{model} loaded from {models_folder} folder')

## 3.1. Feature importances

In [ ]:
# Selected models list
selected_models = ['GBM_train', 'XGB_train', 'rf_train', 'cart_train']

for model in selected_models:
# Plot feature importances
    importances = pd.DataFrame({'value':eval(model).feature_importances_,
                                'variable_name':eval(model).feature_names_in_}).sort_values(by='value', ascending=True)
    fig, ax = plt.subplots(figsize=(10, 7))
    plt.title(f"Feature importances of {eval(model)}")
    plt.barh(importances['variable_name'], importances['value'], color='#00bfc4', align='center')
    plt.xlabel('Relative Feature importance')
    plt.show()

## 3.2. Performance of the models

### Table of main metrics

In [ ]:
# Calculate predictions for all models
y_preds = [eval(model).predict(X_test) for model in models]

# Sensitivity & Specificity
sensitivity = []
specificity = []

for i in range(len(models)):
    tn, fp, fn, tp = confusion_matrix(y_test, y_preds[i]).ravel()
    sensitivity.append(round(tp / (tp + fn), 4))
    specificity.append(round(tn / (tn + fp), 4))

# Dataframe for all metrics, sorted by ROC AUC
metrics_table = pd.DataFrame({
    'Model': [model.split('_')[0].upper() for model in models],
    'ROC AUC': [round(roc_auc_score(y_test, y_preds[i]), 4) for i in range(len(models))],
    'Accuracy': [round(accuracy_score(y_test, y_preds[i]), 4) for i in range(len(models))],
    'Kappa': [round(cohen_kappa_score(y_test, y_preds[i]), 4) for i in range(len(models))],
    'RMSE': [round(mean_squared_error(y_test, y_preds[i]), 4) for i in range(len(models))],
    'MAE': [round(mean_absolute_error(y_test, y_preds[i]), 4) for i in range(len(models))],
    'R2': [round(r2_score(y_test, y_preds[i]), 4) for i in range(len(models))],
    'F1': [round(f1_score(y_test, y_preds[i]), 4) for i in range(len(models))],
    'Sensitivity': sensitivity,
    'Specificity': specificity
}).sort_values(['ROC AUC'], ascending=[False]).style.hide()

# Show table
metrics_table

### Accuracy for Cross Validation (10 splits)

In [ ]:
if file_export_enabled :
    # Calculate scores for accuracy for all models
    kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
    accuracy_scores = []
    for model in models:
        accuracy_scores.append(cross_val_score(eval(model), X_train, y_train, scoring='accuracy', cv=kfold, n_jobs=n_jobs))

    # Store scores in a pandas dataframe and export to external file
    accuracy_scores = pd.DataFrame(accuracy_scores)
    accuracy_scores.columns = accuracy_scores.columns.astype(str)
    accuracy_scores.reset_index().to_feather(datasets_folder + '/' + 'accuracy_scores.feather')

else:
    # Load this dataframe otherwise
    accuracy_scores = pd.read_feather(datasets_folder + '/' + 'accuracy_scores.feather')

In [ ]:
# Boxplots of accuracy values in cross validation
plt.style.use('ggplot')
fig = plt.figure(figsize=(12,5))
fig.suptitle('Cross Validation Accuracy')
ax = fig.add_subplot(111)
plt.boxplot(accuracy_scores[::-1].T.iloc[1:].reset_index(drop=True), vert=False)
ax.set_yticklabels([model.split('_')[0].upper() for model in models][::-1])
plt.show()

#### Statistical hypothesis test

In [ ]:
# Convert score data to list
score_list = accuracy_scores.values.tolist()

# Initiate table
table=[]
for i in range(len(score_list)):
    table.append([])
for i in range(len(score_list)):
    for j in range(len(score_list)):
        table[i].append(0)

# Populate values   
for i in range(len(score_list)): 
    for j in range(i+1,len(score_list)):
        stat, p = ttest_rel(score_list[i], score_list[j])
      
        alpha = 0.05
        
        table[i][j]=np.round(p,3) # upper diagonal
        table[j][i]=np.round(stat,3) # lower diagonal
        table[i][i]=''

print('The following table shows the comparison of the statistics and the p-values of the models')
print('The value of the statistic on the lower diagonal and the p-value on the upper diagonal')

# Show table, including model names
table = pd.DataFrame(table).set_index(pd.Index([model.split('_')[0].upper() for model in models]), inplace=False)
table.columns = [model.split('_')[0].upper() for model in models]
table

## 3.3. Learning curves

In [ ]:
# Function to plot learning curves
def plot_learning_curve(model, X, y, cv, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(f"Learning Curve of {model}")
    plt.xlabel("Training examples")
    plt.ylabel("Score")

    train_sizes, train_scores, test_scores = learning_curve(
        model, X, y, cv=cv, train_sizes=train_sizes, n_jobs=n_jobs)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    return plt

# Set sample split
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=seed)

for model in selected_models:
# Plot learning curves
    plot_learning_curve(eval(model), X_train, y_train, cv=cv)

# 4. Interpretability

In [ ]:
# Create dalex explainers dict for selected models
explainers = {}
for model in selected_models:
    explainers[model] = dx.Explainer(eval(model), X_train, y_train, label=model.split('_')[0].upper())

## 4.1. Residual Diagnostics

In [ ]:
# Plot Residual Diagnostics
for explainer in explainers.values():
    explainer.model_diagnostics().plot()

In [ ]:
# Plot Residual Diagnostics (Absolute)
for explainer in explainers.values():
    explainer.model_diagnostics().plot(variable = "ids", yvariable = "abs_residuals")

## 4.2. Global Explainability - Variable importances

In [ ]:
# Plot variable importances for selected models
for explainer in explainers.values():
    explainer.model_parts().plot()

## 4.3. Partial Dependence Profile (PDP)

In [ ]:
# Plot PDP for selected models
for explainer in explainers.values():
    explainer.model_profile(variables = ["vessel_class_Recreational", "vessel_class_Towing Vessel"] , type = "partial").plot()

## 4.4. Break Down profiles

In [ ]:
# Select a sample observation
vessel_sample = X_train.sample(n=1, random_state=seed)

# Show this observation
vessel_sample

In [ ]:
# Plot shapley values for selected observation
for explainer in explainers.values():
    explainer.predict_parts(new_observation = vessel_sample, type = "break_down").plot()

## 4.5. SHapley Additive exPlanations (SHAP)

In [ ]:
# Calculate predictions (time-consuming process)
if train_model_enabled :
    shaps = {}
    for model in explainers:
        shaps[model] = explainers[model].predict_parts(new_observation = vessel_sample, type = "shap")
    joblib.dump(shaps, models_folder + '/' + 'shaps.pkl')
else:
    shaps = joblib.load(models_folder + '/' + 'shaps.pkl')

# Plot shapley values
for model in explainers:
     shaps[model].plot()

## 4.6. Ceteris Paribus profiles

In [ ]:
# Select a sample observation
vessel_sample = X_train.sample()

# Plot shapley values for selected observation
for explainer in explainers.values():
    explainer.predict_profile(new_observation = vessel_sample).plot(variables = ["vessel_class_Recreational", "vessel_class_Towing Vessel"])

# Session info

In [ ]:
# Python version
import sys
print(f'Python version: {sys.version}')

# Initiate variables for package versions
pkgs_dict = {}
version = []

# Loop importing __version__ for each package (because no whole packages are imported)
for pkg in ['pandas', 'numpy', 'sklearn', 'dalex']:
    exec(f"from {pkg} import __version__ as version")
    pkgs_dict[pkg] = version
    
# Show as table
pd.DataFrame(pkgs_dict, index=["Package version"])

<hr style="border: 1px solid #2fa4e7;">